In [4]:
# ================================================
# 🌟 Quantum Fourier Transform (QFT) Student Practice Tasks
# ================================================

# Make sure pylatexenc is available before drawing
!pip install qiskit qiskit-aer pylatexenc --quiet

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------
# 🧩 Define QFT, Inverse QFT, and Utility Functions
# ------------------------------------------------
def qft_rotations(qc, n):
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

def swap_registers(qc, n):
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

def qft_circuit(n):
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

def inverse_qft_circuit(n):
    qc = QuantumCircuit(n)
    swap_registers(qc, n)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi / 2**(j - m), m, j)
        qc.h(j)
    qc.name = "Inverse QFT"
    return qc


# Helper to safely draw circuits
def safe_draw(qc):
    try:
        display(qc.draw("mpl"))
    except Exception:
        print(qc.draw(output="text"))

# ------------------------------------------------
# 🧠 Task 1: Varying the number of qubits (2, 3, 4)
# ------------------------------------------------
print("🧠 Task 1: Varying the number of qubits (2, 3, 4)")
simulator = AerSimulator(method='statevector')

for n in [2, 3, 4]:
    qc = qft_circuit(n)
    print(f"\n🔹 QFT Circuit for {n} Qubits:")
    safe_draw(qc)
    qc.save_statevector()
    result = simulator.run(transpile(qc, simulator)).result()
    statevector = result.data(0)["statevector"]
    plot_bloch_multivector(statevector)
    plt.show()

# ------------------------------------------------
# 🔄 Task 2: Inverse QFT Implementation
# ------------------------------------------------
print("\n🔄 Task 2: Implementing the Inverse QFT (IQFT)")
n = 3
iqft = inverse_qft_circuit(n)
safe_draw(iqft)

# Verify QFT followed by IQFT returns original state
qc = QuantumCircuit(n)
qc.h(0)
qc.compose(qft_circuit(n), inplace=True)
qc.compose(iqft, inplace=True)
qc.save_statevector()
res = simulator.run(transpile(qc, simulator)).result()
plot_bloch_multivector(res.data(0)["statevector"])
plt.show()

# ------------------------------------------------
# ⚡ Task 3: Integration with Phase Estimation
# ------------------------------------------------
print("\n⚡ Task 3: Combine QFT with a simple Phase Estimation circuit")
n_count = 3
qc = QuantumCircuit(n_count + 1, n_count)
qc.x(n_count)
phi = 0.125
for q in range(n_count):
    qc.cp(2 * np.pi * phi * 2**q, q, n_count)
iqft = inverse_qft_circuit(n_count)
qc.compose(iqft, range(n_count), inplace=True)
qc.measure(range(n_count), range(n_count))
print("🔹 Phase Estimation Circuit:")
safe_draw(qc)
backend = AerSimulator()
counts = backend.run(transpile(qc, backend)).result().get_counts()
plot_histogram(counts)
plt.show()

# ------------------------------------------------
# 🎯 Task 4: Measure Output States of QFT
# ------------------------------------------------
print("\n🎯 Task 4: Measuring the QFT Output State")
n = 3
qc = qft_circuit(n)
qc.measure_all()
sim = AerSimulator()
counts = sim.run(transpile(qc, sim)).result().get_counts()
print("🔹 QFT Circuit with Measurements:")
safe_draw(qc)
plot_histogram(counts)
plt.show()

# ------------------------------------------------
# 🌀 Task 5: Circuit Visualization & Bloch Sphere
# ------------------------------------------------
print("\n🌀 Task 5: Visualizing the Circuit and State on Bloch Sphere")
n = 3
qc = qft_circuit(n)
qc.save_statevector()
res = simulator.run(transpile(qc, simulator)).result()
plot_bloch_multivector(res.data(0)["statevector"])
plt.show()

print("\n✅ All Student Practice Tasks Completed Successfully!")


🧠 Task 1: Varying the number of qubits (2, 3, 4)

🔹 QFT Circuit for 2 Qubits:
                   ┌───┐   
q_0: ──────■───────┤ H ├─X─
     ┌───┐ │P(π/2) └───┘ │ 
q_1: ┤ H ├─■─────────────X─
     └───┘                 

🔹 QFT Circuit for 3 Qubits:
                                          ┌───┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─
           │                ┌───┐ │P(π/2) └───┘ │ 
q_1: ──────┼────────■───────┤ H ├─■─────────────┼─
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │ 
q_2: ┤ H ├─■────────■───────────────────────────X─
     └───┘                                        

🔹 QFT Circuit for 4 Qubits:
                                                                          ┌───┐»
q_0: ──────■───────────────────────────────■──────────────────────■───────┤ H ├»
           │                               │                ┌───┐ │P(π/2) └───┘»
q_1: ──────┼────────■──────────────────────┼────────■───────┤ H ├─■─────────X──»
           │        │                ┌───┐ 